<a href="https://colab.research.google.com/github/itsthemoon/Pytorch-Digit-Identifier-/blob/main/digitClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchinfo

In [39]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt 
from torchinfo import summary

In [ ]:
training = datasets.MNIST("", train=True, download=True, 
                        transform = transforms.Compose([transforms.ToTensor()]))
testing = datasets.MNIST("", train=False, download=True, 
                        transform = transforms.Compose([transforms.ToTensor()]))

In [5]:
train_set = torch.utils.data.DataLoader(training, batch_size=10, shuffle=True)
test_set = torch.utils.data.DataLoader(testing, batch_size=10, shuffle=True)

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    #image 28 x 28 = 784
    self.pool = nn.MaxPool2d(2, 2)
    self.conv1 = nn.Conv2d(1, 10, 5)
    self.conv2 = nn.Conv2d(10, 50, 5)
    self.conv3 = nn.Conv2d(50, 200, 5)
    self.fc = nn.Linear(800,10)


  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = F.relu(self.conv2(x))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1, 800)
    x = self.fc(x)
    x = F.softmax(x,dim=1)
    return x

net = Network()
summary(net,input_size=(1,1,28,28))

In [48]:
network = Network().to(device)
learn_rate = optim.Adam(network.parameters(), lr=0.001)
epochs = 10

In [ ]:
for i in range(epochs):
  for data in train_set:
    image, output = data
    image, output = image.to(device), output.to(device)
    network.zero_grad()
    pred = network(image)
    loss = F.nll_loss(pred, output)
    loss.backward()
    learn_rate.step()
  print(loss.item())

In [ ]:
network.eval() #tells torch we are in evaluate mode (probably need to research this more)
correct = 0
total = 0
with torch.no_grad():
  for data in test_set:
    image, output = data
    image, output = image.to(device), output.to(device)
    result = network(image)
    for index, tensor_value in enumerate(result):
      total += 1
      if torch.argmax(tensor_value) == output[index]:
        correct += 1

accuracy = correct / total 
print(f"Accuracy: {accuracy}")


In [ ]:
#image processing
from PIL import Image
import numpy as np
import PIL.ImageOps   

img = Image.open("test3.png")
img = img.resize((28,28))
img = img.convert("L")
img = PIL.ImageOps.invert(img)

plt.imshow(img)
img = np.array(img)
img = img / 255
image = torch.from_numpy(img)
image = image.float()
image = image.to(device)
#the unsqueeze just makes the 
image = image.unsqueeze(0) 
image = image.unsqueeze(0) 


result = network.forward(image)
print(torch.argmax(result))